open3d 的一些使用样例
---
来源:
- 本地 《2018-Open3D A Modern Library for 3D Data Processing=arxiv1801.09847.pdf》 一些 snippets
- [DEPRECATED] http://www.open3d.org/docs/tutorial/index.html
- [NEW 2019-2-27 16:30:10] http://www.open3d.org/docs/tutorial/Basic/index.html


In [6]:
pwd

u'E:\\Github\\ipython.notebook.dir'

In [1]:
from open3d import *
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
fn_ply = r'E:\3D_Scan\groundtruth-all\12-faces-cut-m.ply'
cloud = read_point_cloud(fn_ply)
print cloud, '***', type(cloud.points)
print cloud.points[0]
print type(cloud.points[0])
print np.asarray(cloud.normals)
print np.asarray(cloud.points)


PointCloud with 5104 points. *** <class 'open3d.open3d.Vector3dVector'>
[ 0.04484    -0.04649    -1.21671999]
<type 'numpy.ndarray'>
[[   5.33884144    3.68257093    8.03490162]
 [  10.25750256    7.87969208   15.12320423]
 [ 105.29602051   77.92811584  164.26605225]
 ..., 
 [  -2.05708051    4.99271107   16.85020447]
 [  -1.80463052    3.21534061   10.9223032 ]
 [  -0.87040818    2.62608075   15.22710419]]
[[ 0.04484    -0.04649    -1.21671999]
 [ 0.04799001 -0.04632    -1.21897006]
 [ 0.05411    -0.06836    -1.21164012]
 ..., 
 [-0.03123    -0.18382001 -1.20454013]
 [-0.0282     -0.18419001 -1.20462   ]
 [-0.0256     -0.18445    -1.20420003]]


In [ ]:
-- Mouse view control --
Left button + drag        : Rotate.
Ctrl + left button + drag : Translate.
Wheel                     : Zoom in/out.

-- Keyboard view control --
[/]          : Increase/decrease field of view.
R            : Reset view point.
Ctrl/Cmd + C : Copy current view status into the clipboard.
Ctrl/Cmd + V : Paste view status from clipboard.

-- General control --
Q, Esc       : Exit window.
H            : Print help message.
P, PrtScn    : Take a screen capture.
D            : Take a depth capture.
O            : Take a capture of current rendering settings.

-- Render mode control --
L            : Turn on/off lighting.
+/-          : Increase/decrease point size.
N            : Turn on/off point cloud normal rendering.
S            : Toggle between mesh flat shading and smooth shading.
W            : Turn on/off mesh wireframe.
B            : Turn on/off back face rendering.
I            : Turn on/off image zoom in interpolation.
T            : Toggle among image render:
               no stretch / keep ratio / freely stretch.

-- Color control --
0..4,9       : Set point cloud color option.
               0 - Default behavior, render point color.
               1 - Render point color.
               2 - x coordinate as color.
               3 - y coordinate as color.
               4 - z coordinate as color.
               9 - normal as color.
Ctrl + 0..4,9: Set mesh color option.
               0 - Default behavior, render uniform gray color.
               1 - Render point color.
               2 - x coordinate as color.
               3 - y coordinate as color.
               4 - z coordinate as color.
               9 - normal as color.
Shift + 0..4 : Color map options.
               0 - Gray scale color.
               1 - JET color map.
               2 - SUMMER color map.
               3 - WINTER color map.
               4 - HOT color map.

In [4]:
'''
notebook 中报错: //可能因为 rdp 远程导致
GLFW Error: WGL: The driver does not appear to support OpenGL
Failed to create window
[DrawGeometries] Failed creating OpenGL window.
不一定，后来本地非rdp也不行，后来本地重启krnl 好了 @2019-2-27 15:51:04
'''

draw_geometries([cloud])


In [7]:
cd E:\Github\Open3D\examples\Python\Basic

E:\Github\Open3D\examples\Python\Basic


In [8]:
# examples/Python/Tutorial/Basic/rgbd_odometry.py

from open3d import *
import numpy as np

if __name__ == "__main__":
    pinhole_camera_intrinsic = read_pinhole_camera_intrinsic(
            "../../TestData/camera_primesense.json")
    print(pinhole_camera_intrinsic.intrinsic_matrix)

    source_color = read_image("../../TestData/RGBD/color/00000.jpg")
    source_depth = read_image("../../TestData/RGBD/depth/00000.png")
    target_color = read_image("../../TestData/RGBD/color/00001.jpg")
    target_depth = read_image("../../TestData/RGBD/depth/00001.png")
    source_rgbd_image = create_rgbd_image_from_color_and_depth(
            source_color, source_depth)
    target_rgbd_image = create_rgbd_image_from_color_and_depth(
            target_color, target_depth)
    target_pcd = create_point_cloud_from_rgbd_image(
            target_rgbd_image, pinhole_camera_intrinsic)

    option = OdometryOption()
    odo_init = np.identity(4)
    print(option)

    [success_color_term, trans_color_term, info] = compute_rgbd_odometry(
            source_rgbd_image, target_rgbd_image,
            pinhole_camera_intrinsic, odo_init,
            RGBDOdometryJacobianFromColorTerm(), option)
    [success_hybrid_term, trans_hybrid_term, info] = compute_rgbd_odometry(
            source_rgbd_image, target_rgbd_image,
            pinhole_camera_intrinsic, odo_init,
            RGBDOdometryJacobianFromHybridTerm(), option) #区别在 Color or Hybrid term
    print '---success_color_term:', success_color_term
    print trans_color_term
    print info

    if success_color_term:
        print("Using RGB-D Odometry")
        print(trans_color_term)
        source_pcd_color_term = create_point_cloud_from_rgbd_image(
                source_rgbd_image, pinhole_camera_intrinsic)
        source_pcd_color_term.transform(trans_color_term)
        draw_geometries([target_pcd, source_pcd_color_term])
    if success_hybrid_term:
        print("Using Hybrid RGB-D Odometry")
        print(trans_hybrid_term)
        source_pcd_hybrid_term = create_point_cloud_from_rgbd_image(
                source_rgbd_image, pinhole_camera_intrinsic)
        source_pcd_hybrid_term.transform(trans_hybrid_term)
        draw_geometries([target_pcd, source_pcd_hybrid_term])


[[ 525.     0.   319.5]
 [   0.   525.   239.5]
 [   0.     0.     1. ]]
OdometryOption class.
iteration_number_per_pyramid_level = [ 20, 10, 5, ] 
max_depth_diff = 0.030000
min_depth = 0.000000
max_depth = 4.000000
---success_color_term: True
[[  9.99988275e-01  -7.36617570e-05  -4.84191029e-03   2.77175781e-04]
 [  1.67413000e-05   9.99930910e-01  -1.17547746e-02   2.29264601e-02]
 [  4.84244164e-03   1.17545557e-02   9.99919187e-01   5.96549283e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ 912274.08573346     983.63060167   39591.10689002       0.
  -454351.99950701  -12920.33039243]
 [    983.63060167  954218.19901262   48468.59479688  454351.99950701
        0.           12945.44672715]
 [  39591.10689002   48468.59479688  134610.38191023   12920.33039243
   -12945.44672715       0.        ]
 [      0.          454351.99950701   12920.33039243  252392.               0.
        0.        ]
 [-454351.99950701       0.          -12945.44672715       

In [9]:
source_rgbd_image

RGBDImage of size 
Color image : 640x480, with 1 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.

In [28]:
'''
@2019-1-17 16:25:18
copy from examples/Python/Tutorial/Basic/rgbd_redwood.py
'''

print("Read Redwood dataset")
color_raw = read_image("E:/Github/Open3D/examples/TestData/RGBD/color/00000.jpg")
depth_raw = read_image("E:/Github/Open3D/examples/TestData/RGBD/depth/00000.png")
rgbd_image = create_rgbd_image_from_color_and_depth(
    color_raw, depth_raw);
print(rgbd_image)
plt.subplot(1, 2, 1)
plt.title('Redwood grayscale image')
plt.imshow(rgbd_image.color)
plt.subplot(1, 2, 2)
plt.title('Redwood depth image')

plt.set_cmap('gray')
plt.imshow(rgbd_image.depth)
plt.show()
pcd = create_point_cloud_from_rgbd_image(rgbd_image, PinholeCameraIntrinsic(
        PinholeCameraIntrinsicParameters.PrimeSenseDefault))
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
draw_geometries([pcd])


Read Redwood dataset
RGBDImage of size 
Color image : 640x480, with 1 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.


In [19]:
set_cmap('jet')

In [15]:
pcd

PointCloud with 267129 points.

In [33]:
'''
# examples/Python/Tutorial/Basic/rgbd_nyu.py
'''

from open3d import *
import numpy as np
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# This is special function used for reading NYU pgm format
# as it is written in big endian byte order.
def read_nyu_pgm(filename, byteorder='>'):
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    img = np.frombuffer(buffer,
        dtype=byteorder+'u2',
        count=int(width)*int(height),
        offset=len(header)).reshape((int(height), int(width)))
    img_out = img.astype('u2')
    return img_out

if __name__ == "__main__":
    print("Read NYU dataset")
    # Open3D does not support ppm/pgm file yet. Not using read_image here.
    # MathplotImage having some ISSUE with NYU pgm file. Not using imread for pgm.
    color_raw = mpimg.imread("E:/Github/Open3D/examples/TestData/RGBD/other_formats/NYU_color.ppm")
    depth_raw = read_nyu_pgm("E:/Github/Open3D/examples/TestData/RGBD/other_formats/NYU_depth.pgm")
    color = Image(color_raw)
    depth = Image(depth_raw)
    rgbd_image = create_rgbd_image_from_nyu_format(color, depth)
    print(rgbd_image)
    plt.subplot(1, 2, 1)
    plt.title('NYU grayscale image')
#     plt.imshow(rgbd_image.color) #c1
#     plt.imshow(color) #c3, ok
    plt.imshow(color_raw) #c3, ok
    plt.subplot(1, 2, 2)
    plt.title('NYU depth image')
    plt.imshow(rgbd_image.depth)
    plt.show()
    pcd = create_point_cloud_from_rgbd_image(rgbd_image, PinholeCameraIntrinsic(
            PinholeCameraIntrinsicParameters.PrimeSenseDefault))
    # Flip it, otherwise the pointcloud will be upside down
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    draw_geometries([pcd])


Read NYU dataset
RGBDImage of size 
Color image : 640x480, with 1 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.


In [36]:
Image

pybind11_builtins.pybind11_type

In [37]:
plt.imshow(color_raw)

In [25]:
__name__

'__main__'

In [17]:
draw_geometries([source_pcd_color_term,])

In [13]:
source_rgbd_image.

RGBDImage of size 
Color image : 640x480, with 1 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.